# Classifying Images

(WORKING ON TENSORFLOW >2.0 )

Classify images using convolutional NN.


In [ ]:
# Load libraries

import tensorflow as tf
import numpy as np
from keras import backend as K 
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, Convolution2D

In [ ]:
# Set the color channel value will be first

K.set_image_data_format("channels_first")

In [ ]:
# Seed Seeds

np.random.seed(0)

In [ ]:
# Set image information

channels = 1
height = 28
width = 28
shape = (1,28, 28)

In [ ]:
# Load data and target from MNIST data

(data_train, target_train), (data_test, target_test) = mnist.load_data()

In [ ]:
# Reshape training image data into features

data_train = data_train.reshape(data_train.shape[0], channels, height, width)

In [ ]:
# Reshape test image data into features

data_test = data_test.reshape(data_test.shape[0], channels, height, width)

In [ ]:
# Rescale pixel intensity to between 0 and 1 

features_train = data_train / 255
features_test = data_test / 255

In [ ]:
# One hot encode target

target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_classes = target_test.shape[1]

In [ ]:
# Start NN 

network = Sequential()

In [ ]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback


print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus()
tfback._get_available_gpus

In [ ]:
# Add convolutional layer with 64 filters, 5x5 window and relu activation

network.add(Conv2D(filters = 64, strides = (1,1), padding='valid', kernel_size = (5,5), input_shape = (channels, width, height) , activation = "relu"))

In [ ]:
# Add max pooling layer with a 2x2 window

network.add(Dropout(0.5))

In [ ]:
# Add layer to flatten input

network.add(Flatten())

In [ ]:
# Add fully connected layer of º28 units with a ReLu activation function

network.add(Dense(128, activation = "relu"))

In [ ]:
# Add dropout layer

network.add(Dropout(0.5))

In [ ]:
# Add fully connected layer of º28 units with a ReLu activation function

network.add(Dense(number_classes, activation = "softmax"))

In [ ]:
# Compile NN

network.compile(loss = "categorical_crossentropy", optimizer = "rmsprop", metrics = ["accuracy"])

In [ ]:
# Train NN 

network.fit(features_train, target_train, epochs= 2, verbose = 1, batch_size = 1000, validation_data = (features_test, target_test))

# Improving performance with image augmentation



In [ ]:
# Load library

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Creatly rotatee image augmentation

augmentation = ImageDataGenerator(featurewise_center = True, #Apply ZCA whitening
                                  zoom_range = 0.3, # Randomly zoom in on images
                                  width_shift_range = 0.2, # Randomly shift images
                                  horizontal_flip = True, # Randomly flip images
                                  rotation_range = 90) # Randomly rotate

In [ ]:
# Process all images from the directory "raw/images"

augment_images = augmentation.flow_from_directory("/",
                                                  batch_size = 32, 
                                                  class_mode = "binary",
                                                  save_to_dir = "/")

In [ ]:

network.fit_generator(augment_images,
                      steps_per_epoch = 2000, # Number of times to call the generator foe each epoch
                      epochs = 5, 
                      validation_data = augment_images,# Test data generator
                      validation_steps = 800) # Number of items to call the generator for each test epoch